In [1]:
import tweepy

consumer_key = '...'
consumer_secret = '...'
access_token = '...'
access_token_secret = '...'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

#Comprobar que funciona la api
status = api.get_status('937349434668498944')
print(status.user.id)
user = api.get_user(status.user.id).screen_name
api.get_user('Humbert18960727').created_at

4219197432


datetime.datetime(2016, 1, 6, 16, 42, 23)

In [2]:
import time
import tweepy
import pandas as pd
import matplotlib.pyplot as plt

def get_tweet_status(tweet_id):
    '''
    Devuelve un objeto status con todos los datos de un tweet
    '''
    try:
        return api.get_status(tweet_id)
    except tweepy.TweepError as e:
        return 'SuspendedAccount'

def get_user_by_id(id):
    '''
    Devuelve un objeto user a traves del id del user
    '''
    return api.get_user(id)

def get_screen_name(id):
    try:
        user = get_user_by_id(id)
        screen_name = user.screen_name
        return screen_name
    except tweepy.TweepError as e:
        return ('SuspendedAccount')    
    
def get_screen_name_from_tweet(tweet_id):
    try:
        status = api.get_status(tweet_id)
        user = get_user_by_id(status.user.id)
        screen_name = user.screen_name
        return screen_name
    except tweepy.TweepError as e:
        return ('SuspendedAccount')    

def get_followers_page(screen_name):
    '''
    Devuelve los ids de los usuarios que sigue un user, screen_name es el nombre de usuario sin el '@'
    ej @NelsonMandela -> NelsonMandela
    '''
    ids = []
    for page in tweepy.Cursor(api.followers_ids, screen_name=screen_name).pages():
        ids.extend(page)
        #time.sleep(60)
    return ids



def get_followers_limited(screen_name, total):
    '''
    Devuelve los ids de los usuarios que sigue un user, screen_name es el nombre de usuario sin el '@'
    ej @NelsonMandela -> NelsonMandela
    Obtiene solo un numero de ids de usuarios indicado en el parametro total
    '''
    ids = []
    for page in tweepy.Cursor(api.followers_ids, screen_name=screen_name).items(total):
        ids.append(page)
    return ids

def get_followers_from_tweet_id(tweet_id, total = 0):
    '''
    Pasandole el id de un tweet obtenemos todos sus seguidores,
    se le puede pasar el parametro total y obtener solamente un numero concreto de ids
    '''
    user_id = get_tweet_status(tweet_id).user.id
    screen_name = get_user_by_id(user_id).screen_name
    if total == 0:
        followers_ids = get_followers_page(screen_name)
    else:
        followers_ids = get_followers_limited(screen_name,total)
    return followers_ids
    
    

def get_tweets(num_noticias):
    '''
    Le pasamos el numero de noticias de las cuales queremos obtener los tweets_ids
    Con el DataFrame de Pandas separamos los tweets de una misma noticia
    Devuelve un array de [titulos, array(tweets_ids)]
    '''
    df = pd.read_csv('./data/politifact_fake.csv') 
    #Esto se puede cambiar por otro dataset (ej: ./data/politifact_real.csv), pero de momento trabajamos con este
    df['tweet_ids'][0].split('\t')
    
    titulo = []
    tweets = []
    i = 0
    
    for tweet in df['tweet_ids']:
        if i < num_noticias:
            array_tweets = str(tweet).split('\t')
            #Se puede cambiar a que tenga al menos 5 o 10 tweets para probar
            if(len(array_tweets) > 1 and len(array_tweets) < 2000): #Solo si tiene algun tweet lo añadimos, 1 elemento es NaN
                tweets.append(array_tweets)
                titulo.append(df['title'][i])
                i = i+1
    noticias = [titulo, tweets]
    return noticias 

# Grafo de propagación de noticias 

Este grafo sirve para ver la difusión de la noticia, es decir, ver las veces que ha sido retweeteado, para ello lo separamos en nivels: 
nivel 0: la noticia como origen. 

nivel 1: las personas que han tweeado sobre la noticia del nivel 1.

nivel 2: aquellos que han hecho RT del tweet del nivel 1.

nivel 3: todos que han hecho RT del RT del nivel 2. 

Asi sucesivamete hasta que nadie haga mas RT, ahi es cuando se habrá terminado la difusión. 

In [5]:
# Usado solo para sacar dos noticis 

#Sacamos el nivel 0 = la noticia

df = pd.read_csv('./data/politifact_fake.csv',delimiter=",")
df4 = df.tail(2).copy()


#id_noticia = df['id'][0]
#id_noticia = df.loc[0:3,'id']


#list_nivel1 = df.loc[0,'tweet_ids'].split('\t')

#politic_news = get_tweets(4)
titulo = []
tweets = []
noticias= []
i = 0
for tweet in df4['tweet_ids']:
    array_tweets = str(tweet).split('\t')
    if(len(array_tweets) > 1 and len(array_tweets) < 2000): #Solo si tiene algun tweet lo añadimos, 1 elemento es NaN
                tweets.append(array_tweets)
                titulo.append(df['title'][i])
                i = i+1
    noticias = [titulo, tweets]
    

In [6]:
# ids de noticisa convertidos en una lista
ids = []

for i in range (0,4): 
    ids.append(noticias[1][1][i])

In [268]:
nodos

,padre,hijos
0,0,"[805216591424212993, 805222830023540736, 80522..."
1,1,"[934217740692443136, 934217839816445952, 93421..."


In [3]:
def get_ids_fromStatuses(statuses):
     '''
    Dado un objeto statuses de un tweet nos devuelve la lista de ids que han rtweeteado ese tweet(statuses)
    '''
    lis_ids = []
    
    for i in range (len(statuses)): 
        lis_ids.append(statuses[i].id)
    return lis_ids

In [4]:
def get_nodo(list_id):
    '''
    le pasamos una lista de ids de tweets y nos devuelve por cada ids los ids de los rts en un DataFrame
    '''
    nodos = pd.DataFrame(columns = ['padre', 'hijos'])
    
    for i in range (len(list_id)):
        t = api.retweets(list_id[i])
        if t != 'SuspendedAccount':
            l_rts = get_ids_fromStatuses(t)
            nodos = nodos.append({"padre": list_id[i], "hijos": l_rts}, ignore_index=True)
            
    return nodos

In [54]:
def get_propagacion_graph(ids0): #Es prueba solo para una lista de ids previamente convertidos
    grafo = pd.DataFrame(columns = ['padre', 'hijos'])
    grafo = grafo.append(get_nodo(ids0), ignore_index=True)
    grafo_aux = pd.DataFrame(columns = ['padre', 'hijos'])
    for i in range (grafo["padre"].count()):
        grafo_aux = grafo_aux.append(get_nodo(grafo["hijos"][i]),ignore_index=True)
    
    grafo = grafo.append(grafo_aux,ignore_index=True)
    return grafo

In [122]:
def get_propagacionGraph(): 
      '''
    nos saca el grafo de una noticia a partir de una lista de ids sacados del csv politicalfact.csv
    Para sacar mas grafos de mas noticias de golpe solo habria que pasar un argumento a la funcion con la lista de ids de
    cada noticia y hacer una estructura iterativa en otra funcion, fuera, que llame a esta funcion tantas veces como queramos 
    
    tambine se podria desde esta misma funcin leyendo el csv dentro e iterando tantas veces como noticias haya. 
    '''
    grafo = pd.DataFrame(columns = ['padre', 'hijos'])
    grafo = grafo.append(get_nodo(ids), ignore_index=True)
    
    etapa = get_next_level(grafo)
    
    #grafo = grafo.append(etapa,ignore_index=True)
        
    return etapa[0]
    

In [123]:
def get_next_level(df_ph): 
    '''
    esta funcion nos permite sacar los nodos del nivel anterios, devuelve una tupla de df [0] es el grafo resultante, 
    [1] es el nivel anterior y que vamos a sacar sus hijos se realiza de forma recursiva. 
    '''
    grafo_aux = pd.DataFrame(columns = ['padre', 'hijos'])
    grafo_aut = pd.DataFrame(columns = ['padre', 'hijos'])
    grafo_aut = df_ph.copy()
    for i in range (grafo_aut["padre"].count()):
        if bool(grafo_aut.iloc[:,1][i]) == True: #comprueba que la lisat de hijos no este vacia para llamar getNodo sino continua 
            grafo_aux = grafo_aux.append(get_nodo(grafo_aut["hijos"][i]),ignore_index=True)
    
    if grafo_aux["hijos"].count() > 0: #comprueba que no este vacio para hacer la llamada recursiva 
        get_next_level(grafo_aux)
    
    grafo_aut = grafo_aut.append(grafo_aux,ignore_index=True)
        
    return [grafo_aut,grafo_aux]

In [124]:
p = get_propagacionGraph()

In [125]:
p

,padre,hijos
0,934217740692443136,"[934251194620547078, 934244963612639233, 93422..."
1,934217839816445952,[]
2,934217896355561473,"[934433718793637888, 934412241335672832, 93441..."
3,934217997006376960,"[934419247203315713, 934418794491056129]"
4,934251194620547078,[]
5,934244963612639233,[]
6,934221180474658816,[]
7,934219417042456577,[]
8,934218079504125952,[]
9,934217919097122818,[]
